한국어 영화 리뷰 - BERT

In [2]:
import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)

In [11]:
# rating 6이상이면 긍정  라벨 생성
import numpy as np
from sklearn.model_selection import train_test_split
# y를 생성해서 review컬럼이 x  데이터 분할  데이터 개수 확인
y = df.rating.apply(lambda x: 1 if x >=6 else 0)
X = df.review
# 데이터 셋을 학습 검증 평가로 나눈다
X_, x_test, y_, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42, stratify=y_)
print(x_train.shape, x_val.shape, x_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(9424,) (2356,) (2945,)
(9424,) (2356,) (2945,)


In [21]:
import torch
# from datatsets import load_metric # 2022 이후로 HugginFace에서 deprecated  evaluate
# old system -> legacy system
import evaluate
metric = evaluate.load('accuracy')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
  '''
  Args:
    eval_pred : logits.labels를 가지고 있는 datasets
  Returns:
    accuracy
  '''
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
# 데이터셋 생성  클래스(상속)
# __init__  __getitem__  __len__
# x,y        각각 텐서    y의 개수
class OurDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels